In [1]:
#Importando as bibliotecas necessárias para analisar os dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import openpyxl.utils.cell as cell


#import seaborn as sns




KeyboardInterrupt



In [ ]:
#Importando os dados
data_frame = pd.read_excel('BANCO DE DADOS - INVERTIDO - 24 de Agosto 2019.xlsx')

def number_to_excel_column(num):
    """
    Converte um número de coluna em uma string com a representação alfabética.
    Exemplo: 1 => 'A', 2 => 'B', ..., 26 => 'Z', 27 => 'AA', 28 => 'AB', ...
    """
    return cell.get_column_letter(num)

# Gera uma lista com os nomes das colunas no estilo do Excel
col_names = [number_to_excel_column(num) for num in range(1, len(data_frame.columns)+1)]

# Cria um dicionário com os nomes atuais e os novos nomes das colunas
new_col_names = {old_name: new_name for old_name, new_name in zip(data_frame.columns, col_names)}

# Renomeia as colunas
data_frame = data_frame.rename(columns=new_col_names)

# Cria uma lista com os nomes das colunas que devem ser removidas
col_names = []

data_frame_r = data_frame.replace('', np.nan)


# Parte do dataframe que contem o questionário de "Social"
df_socioeconomico = data_frame_r.loc[:, 'B':'BP']
# Somar colunas que envolvem o uso de drogas
df_socioeconomico = df_socioeconomico.assign(USO_DROGAS=df_socioeconomico.loc[:, 'BF':'BN'].sum(axis=1))

# Parte do dataframe que contem o questionário WHOQOL
df_whoqol = data_frame_r.loc[:, 'BR':'CU']

# Parte do dataframe que contem o questionário de MHI
df_mhi = data_frame_r.loc[:, 'CW':'EH']

# Juntando os dataframes
df = pd.concat([df_socioeconomico, df_whoqol, df_mhi], axis=1)

# Pegar a coluna que tem o curso de cada aluno que será para ser previsto
df_curso = df.loc[:, 'K']
df = df.drop(columns=['K'])

# Concatenando o dataframe com o curso
df = pd.concat([df, df_curso], axis=1)


In [ ]:


# Veficicando Tamnho do dataframe
print(df.shape)

# Checando se há valores nulos
#print(df.isnull().sum())

# Removendo linhas com valores nulos
df = df.dropna()

# Checando quantos elementos foram removidos
print('Shape',df.shape)

# Separando os dados de X e y
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values



#unique, counts = np.unique(y, return_counts=True)
#print(np.asarray((unique, counts)).T)


In [ ]:
# Realizando a normalização dos dados
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Separando os dados de treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o modelo de classificação simples e rapido
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model.predict(X_test)

# Avaliando o modelo
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


# Modelo utilizando o KNN
from sklearn.neighbors import KNeighborsClassifier
model_knn = KNeighborsClassifier(n_neighbors=5)
model_knn.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_knn.predict(X_test)

# Avaliando o modelo
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))


# Modelo utilizando o SVM
from sklearn.svm import SVC
model_svm = SVC(kernel='linear', random_state=42, decision_function_shape='ovo')
model_svm.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_svm.predict(X_test)

# Avaliando o modelo
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

In [ ]:
# Implementando grid search do PCA e SVM para encontrar os atributos e parâmetros ideais
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pipe = Pipeline([('pca', PCA()), ('svm', SVC(decision_function_shape='ovo'))])

param_grid = [
    {
        'pca__n_components': np.arange(45, np.shape(X)[1]),
        'svm__C': [0.1, 1, 10],
        'svm__kernel': ['linear','rbf']
    }
]

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)
print(grid.score(X_test, y_test))

# Selecionando melhores atributos do resultado do grid search
pca = PCA(n_components=grid.best_params_['pca__n_components'])
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Criando modelo SVM com os melhores parâmetros
model_svm = SVC(kernel=grid.best_params_['svm__kernel'], C=grid.best_params_['svm__C'], random_state=42)
model_svm.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_svm.predict(X_test)

# Avaliando modelo com metricas relevantes
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))




In [ ]:
# # Modelo utilizando o XGBoost
# from xgboost import XGBClassifier
# model_xgb = XGBClassifier()
# model_xgb.fit(X_train, y_train)

# # Fazendo as previsões
# y_pred = model_xgb.predict(X_test)

# # Avaliando o modelo
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# print(accuracy_score(y_test, y_pred))
# #print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

# Modelo utilizando o LightGBM
from lightgbm import LGBMClassifier
model_lgbm = LGBMClassifier()
model_lgbm.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_lgbm.predict(X_test)

# Avaliando o modelo
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))


# Modelo utilizando o MLP
from sklearn.neural_network import MLPClassifier
model_mlp = MLPClassifier()
model_mlp.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_mlp.predict(X_test)

# Avaliando o modelo
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))


# Modelo que seleciona aleatoriamente a classe a fim de comparar com os modelos acima
from sklearn.dummy import DummyClassifier
model_dummy = DummyClassifier(strategy='stratified')
model_dummy.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_dummy.predict(X_test)

# Avaliando o modelo
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))


In [ ]:
# Implementando Grid Seach com MLP
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'hidden_layer_sizes': [(100, 100, 100), (100, 100), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

mlp = MLPClassifier(max_iter=1000)
clf = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1)
clf.fit(X_train, y_train)
print(clf.best_params_)

# Criando modelo MLP com os melhores parâmetros
model_mlp = MLPClassifier(hidden_layer_sizes=clf.best_params_['hidden_layer_sizes'], activation=clf.best_params_['activation'], solver=clf.best_params_['solver'], alpha=clf.best_params_['alpha'], learning_rate=clf.best_params_['learning_rate'], max_iter=1000)
model_mlp.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_mlp.predict(X_test)

# Avaliando modelo com metricas relevantes
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))
#print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# Implementando AutoML
from tpot import TPOTClassifier
tpot = TPOTClassifier(verbosity=2, n_jobs=-1)
tpot.fit(X_train, y_train)
tpot.export('tpot_pipeline.py')
print(tpot.score(X_test, y_test))


# Fazendo as previsões
y_pred = tpot.predict(X_test)

# Avaliando modelo com metricas relevantes
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))


In [ ]:
# Pegar melhor modelo do AutoML[
print(tpot.fitted_pipeline_)
print(tpot.fitted_pipeline_.steps[0][1].get_params())
#print(tpot.fitted_pipeline_.steps[1][1].get_params())

# Testando modelo com os melhores parâmetros do AutoML
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(bootstrap=tpot.fitted_pipeline_.steps[0][1].get_params()['bootstrap'], max_features=tpot.fitted_pipeline_.steps[0][1].get_params()['max_features'], min_samples_leaf=tpot.fitted_pipeline_.steps[0][1].get_params()['min_samples_leaf'], min_samples_split=tpot.fitted_pipeline_.steps[0][1].get_params()['min_samples_split'], n_estimators=tpot.fitted_pipeline_.steps[0][1].get_params()['n_estimators'])
model_rf.fit(X_train, y_train)

# Fazendo as previsões
y_pred = model_rf.predict(X_test)

# Avaliando modelo com metricas relevantes
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test, y_pred))